In [11]:
%matplotlib inline
import torch
import os,sys
import numpy as np
import torchvision.io as io
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from torchvision import transforms 
from torch.utils.data import Dataset, random_split

In [27]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [3]:
if "google.colab" in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True, use_metadata_server=False)

Mounted at /content/drive


In [ ]:
def import_file(url, filename):
  import requests
  r = requests.get(url)

  with open(filename, 'w') as f:
        f.write(r.text)

if "google.colab" in sys.modules:
    # Save datagenerators as file to colab working directory
    # If you are using GitHub, make sure you get the "Raw" version of the code
    url_u_net = 'https://raw.githubusercontent.com/karimassi/road-segmentation/u_net/src/models/u_net.py?token=AJVSYDCYFSNCXQW2B5UOPG27ZJREK'
    import_file(url_u_net, 'u_net.py')

    url_u_net_paper = 'https://raw.githubusercontent.com/karimassi/road-segmentation/u_net/src/models/u_net_paper.py?token=AJVSYDBJJBS6FXDVBFXKV7S7ZJRGY'
    import_file(url_u_net_paper, 'u_net_paper.py')

    url_training = 'https://raw.githubusercontent.com/karimassi/road-segmentation/u_net/src/training.py?token=AJVSYDBJO3AY5WONIKOAU4K7Z7QIO'
    import_file(url_training, 'training.py')

    url_image_mask_dataset = "https://raw.githubusercontent.com/karimassi/road-segmentation/u_net/src/image_mask_dataset.py?token=AJVSYDDNI2HGBE2YKVRZXFK7ZWOBA"
    import_file(url_image_mask_dataset, 'image_mask_dataset.py')
        
    import u_net
    import u_net_paper 
    import training
    import image_mask_dataset
else:
    from src import u_net
    from src import u_net_paper
    from src import training
    from src import image_mask_dataset

Mounted at /content/drive


In [4]:
# load data
import image_mask_dataset

root_dir = "/content/drive/Shareddrives/road-segmentation/data/training/"
image_dir = root_dir + "images/"
gt_dir = root_dir + "groundtruth/"

dataset = image_mask_dataset.ImageMaskDataset(image_dir, gt_dir)

In [52]:
num_epochs = 10
learning_rate = 1e-5
batch_size = 5

data_len = len(dataset)
train_len = int(data_len * 0.9)
test_len = int(data_len * 0.1)

dataset_train, dataset_test = random_split(dataset, [train_len, test_len])
print(len(dataset_train), len(dataset_test))

dataloader_train = torch.utils.data.DataLoader(
    dataset_train,
    batch_size=batch_size,
    shuffle=True
)

dataloader_test = torch.utils.data.DataLoader(
    dataset_test,
    batch_size=batch_size,
    shuffle=True
)

import u_net_paper
from u_net_paper import UNet_paper

NUM_CHANNELS = 3
NUM_FILTERS = 32

import training

# Train the u-net model with the SGD optimiz
model = UNet_paper(NUM_CHANNELS, NUM_FILTERS).to(device)

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
training.train(model, dataloader_train, dataloader_test, optimizer, num_epochs)

#from torchsummary import summary
#summary(model, input_size=(NUM_CHANNELS, 400, 400))

90 10
Starting training
Epoch  1 | Test accuracy : 0.0 | Test F1 : 0.46024
Epoch  2 | Test accuracy : 0.0 | Test F1 : 0.46257
Epoch  3 | Test accuracy : 0.0 | Test F1 : 0.46418
Epoch  4 | Test accuracy : 0.0 | Test F1 : 0.46454
Epoch  5 | Test accuracy : 0.0 | Test F1 : 0.46423
Epoch  6 | Test accuracy : 0.0 | Test F1 : 0.46412
Epoch  7 | Test accuracy : 0.0 | Test F1 : 0.46457
Epoch  8 | Test accuracy : 0.0 | Test F1 : 0.46461
Epoch  9 | Test accuracy : 0.0 | Test F1 : 0.46442
Epoch  10 | Test accuracy : 0.0 | Test F1 : 0.46459
